In [11]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import os

device = "cuda" if torch.cuda.is_available() else "cpu"


In [12]:
train_df = pd.read_csv("/kaggle/input/bean-leaf-lesions-classification/train.csv")
val_df = pd.read_csv("/kaggle/input/bean-leaf-lesions-classification/val.csv")

transform = transforms.Compose([
    transforms.Resize([128,128]),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

In [13]:
# Global dataset base path
base_dir = "/kaggle/input/bean-leaf-lesions-classification/"

class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.labels = torch.tensor(dataframe["category"]).long()

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_rel_path = self.dataframe.iloc[idx, 0]  # e.g., "train/healthy/file.jpg"
        img_path = base_dir + img_rel_path  # use global base_dir

        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [14]:
train_dataset = CustomImageDataset(dataframe = train_df, transform = transform)
val_dataset = CustomImageDataset(dataframe = val_df, transform = transform)

lr = 1e-3
batch_size = 4
epochs = 15
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)

googlenet_model = models.googlenet(weights = "DEFAULT")

for param in googlenet_model.parameters():
    param.requires_grad = True

num_classes = len(train_df["category"].unique())
googlenet_model.fc = torch.nn.Linear(googlenet_model.fc.in_features, num_classes)
googlenet_model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [15]:
loss_fun = nn.CrossEntropyLoss()
optimizer = Adam(googlenet_model.parameters(), lr = lr)
total_loss_train_plot = []
total_acc_train_plot = []

for epoch in range(epochs):
    total_acc_train = 0
    total_loss_train = 0

    googlenet_model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  
        optimizer.zero_grad()
        outputs = googlenet_model(inputs)
        train_loss = loss_fun(outputs, labels)
        total_loss_train += train_loss.item()

        train_loss.backward()
        optimizer.step()

        total_acc_train += (torch.argmax(outputs, dim=1) == labels).sum().item()

    avg_loss = round(total_loss_train / len(train_loader), 4)
    avg_acc = round(total_acc_train / len(train_dataset) * 100, 4)

    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss}, Accuracy: {avg_acc}%")


Epoch 1/15 - Loss: 0.7943, Accuracy: 67.5048%
Epoch 2/15 - Loss: 0.6176, Accuracy: 76.0155%
Epoch 3/15 - Loss: 0.5319, Accuracy: 80.6576%
Epoch 4/15 - Loss: 0.4155, Accuracy: 85.7834%
Epoch 5/15 - Loss: 0.3743, Accuracy: 87.234%
Epoch 6/15 - Loss: 0.3201, Accuracy: 89.0716%
Epoch 7/15 - Loss: 0.2765, Accuracy: 91.0058%
Epoch 8/15 - Loss: 0.31, Accuracy: 88.7814%
Epoch 9/15 - Loss: 0.2202, Accuracy: 91.7795%
Epoch 10/15 - Loss: 0.2934, Accuracy: 90.1354%
Epoch 11/15 - Loss: 0.2602, Accuracy: 92.0696%
Epoch 12/15 - Loss: 0.1633, Accuracy: 94.4874%
Epoch 13/15 - Loss: 0.1279, Accuracy: 95.5513%
Epoch 14/15 - Loss: 0.1282, Accuracy: 96.5184%
Epoch 15/15 - Loss: 0.1575, Accuracy: 94.4874%


In [20]:
with torch.no_grad():
    total_acc_test = 0
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)  
        prediction = googlenet_model(inputs)
        acc = (torch.argmax(prediction, axis =1) == labels).sum().item()
        total_acc_test += acc

print("Testing Accuracy", round(total_acc_test/val_dataset.__len__()*100, 2))

Testing Accuracy 92.48
